In [1]:
#importing libraries

import tensorflow as tf
import pandas as pd
import numpy as np
import os
from PIL import Image
from scipy.misc import imread, imsave, imresize
import heapq

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
#converting all to RGB

img_list = []
img_names = os.listdir("train/train")
for img_name in (os.listdir("train/train")):
    img_list.append("train/train/" + img_name)
    
i = 0
for img in img_list:
    img = Image.open(img)
    rgbimg = Image.new("RGB", img.size)
    rgbimg.paste(img)
    rgbimg.save('new_train/{}'.format(img_names[i]))
    i += 1    
    
img_list = []
img_names = os.listdir("test/test")
for img_name in (os.listdir("test/test")):
    img_list.append("test/test/" + img_name)
    
i = 0
for img in img_list:
    img = Image.open(img)
    rgbimg = Image.new("RGB", img.size)
    rgbimg.paste(img)
    rgbimg.save('new_test/{}'.format(img_names[i]))
    i += 1    

In [2]:
#getting output labels

def get_labels(predictions):
    
    input_train_file = "train.csv"
    train_dataframe = pd.read_csv(input_train_file, header = 0)
    train_labels = train_dataframe["Id"].as_matrix()

    labels_dict = {}
    counter = 0
    for i in range(9850):
        if train_labels[i] in labels_dict:
            labels_dict[train_labels[i]][1] += 1
        else:
            labels_dict[train_labels[i]] = [counter, 1]
            counter += 1
            
    largest = heapq.nlargest(5, predictions) 
    
    largest_indices = []
    
    for i in range(5):
        largest_indices.append(list(predictions).index(largest[i]))
        
    true_labels = []
    
    for i in range(5):
        for label in labels_dict:
            if labels_dict[label][0] == largest_indices[i]:
                true_labels.append(label)
                
    return true_labels            

In [3]:
#getting train data

def get_data():
    
    input_train_file = "train.csv"
    train_dataframe = pd.read_csv(input_train_file, header = 0)
    train_labels = train_dataframe["Id"].as_matrix()

    labels_dict = {}
    counter = 0
    for i in range(9850):
        if train_labels[i] in labels_dict:
            labels_dict[train_labels[i]][1] += 1
        else:
            labels_dict[train_labels[i]] = [counter, 1]
            counter += 1

    img_list = []
    for img_name in (os.listdir("new_train")):
        img_list.append("new_train/" + img_name)

    indices_list = []
    for j in range(9850):
        indices_list.append(labels_dict[train_labels[j]][0])

    one_hot_labels = tf.one_hot(indices_list, 4251) 

    with tf.Session() as sess:
        one_hot_labels = one_hot_labels.eval()
    
    def _parse_function(filename, label):
        image_string = tf.read_file(filename)
        image_decoded = tf.image.decode_jpeg(image_string)
        image_resized = tf.image.resize_images(image_decoded, [512, 512])
        return image_resized, label

    filenames = tf.constant(img_list)

    labels = tf.constant(indices_list)

    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

    dataset = dataset.map(_parse_function)
    
    return dataset, one_hot_labels

In [4]:
#getting test data

def get_test_data():
    
    img_list = []
    for img_name in (os.listdir("new_test")):
        img_list.append("new_test/" + img_name)
    
    def _parse_function(filename):
        image_string = tf.read_file(filename)
        image_decoded = tf.image.decode_jpeg(image_string)
        image_resized = tf.image.resize_images(image_decoded, [512, 512])
        return image_resized
    
    filenames = tf.constant(img_list)
    
    dataset = tf.data.Dataset.from_tensor_slices((filenames))

    dataset = dataset.map(_parse_function)
    
    return dataset

In [5]:
#creating the model 

def model():
    
    features = tf.placeholder(tf.float32, shape = (50, 512, 512, 3), name = "features")
    labels = tf.placeholder(tf.float32, shape = (50, 4251), name = "labels")
    
    input_layer = tf.reshape(features, (50, 512, 512, 3))
    
    conv1 = tf.contrib.layers.conv2d(
    inputs = input_layer,
    num_outputs = 16,
    kernel_size = [3 , 3],
    stride = 1,
    padding = 'SAME')
    
    pool1 = tf.contrib.layers.max_pool2d(
    inputs = conv1,
    kernel_size = [3 , 3],
    stride = 1,
    padding = 'VALID')
    
    conv2 = tf.contrib.layers.conv2d(
    inputs = pool1,
    num_outputs = 32,
    kernel_size = [5 , 5],
    stride = 3,
    padding = 'SAME')
    
    pool2 = tf.contrib.layers.max_pool2d(
    inputs = conv2,
    kernel_size = [3 , 3],
    stride = 1,
    padding = 'VALID')
    
    conv3 = tf.contrib.layers.conv2d(
    inputs = pool2,
    num_outputs = 64,
    kernel_size = [7 , 7],
    stride = 5,
    padding = 'SAME')
    
    pool3 = tf.contrib.layers.max_pool2d(
    inputs = conv3,
    kernel_size = [3 , 3],
    stride = 1,
    padding = 'VALID')
    
    conv4 = tf.contrib.layers.conv2d(
    inputs = pool3,
    num_outputs = 128,
    kernel_size = [9 , 9],
    stride = 7,
    padding = 'SAME')
    
    pool4 = tf.contrib.layers.max_pool2d(
    inputs = conv4,
    kernel_size = [3 , 3],
    stride = 1,
    padding = 'VALID')
    
    flatten_layer = tf.contrib.layers.flatten(pool4)
    
    fc1 = tf.contrib.layers.fully_connected(flatten_layer, 10000)
    fc2 = tf.contrib.layers.dropout(fc1, keep_prob = 0.8)
    fc3 = tf.contrib.layers.fully_connected(fc2, 6000)
    pred = tf.contrib.layers.fully_connected(fc3, 4251)
    
    loss = tf.losses.softmax_cross_entropy(labels, pred)
    
    return features, labels, pred, loss

In [6]:
#running the model

def run_model(num_epochs):
    
    test_img_list =  os.listdir("new_test")
    
    dataset1, one_hot_labels = get_data()
    
    features, labels, pred, loss = model()
    
    dataset1 = dataset1.repeat().batch(50)
    
    iter = dataset1.make_one_shot_iterator()
    
    x, y = iter.get_next()
    
    dataset2 = get_test_data()
    
    dataset2 = dataset2.repeat().batch(50)
    
    iter = dataset2.make_one_shot_iterator()
    
    x_test = iter.get_next()
         
    optimizer = tf.train.MomentumOptimizer(0.001, 0.9).minimize(loss)
    
    init = tf.global_variables_initializer()
    
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        
        sess.run(init)
        for i in range(num_epochs):
            for j in range(197):
                new_x =x.eval()
                feed_dict = {features: new_x, labels: one_hot_labels[j * 50 : (j+1) * 50][:]}
                _, loss_value = sess.run([optimizer, loss], feed_dict)
                print("Epoch: {}, Iter: {}, Loss: {:.4f}".format(i, j, loss_value))
        
        save_path = saver.save(sess, "model/hbw_model.ckpt")
        
        print("Model saved in path: %s" % save_path)
        
        df = pd.DataFrame([], columns=['Image', ' Id'])
        m = 0
        
        for j in range(313):
            new_x_test = x_test.eval()
            feed_dict_test = {features: new_x_test}
            prediction = sess.run(pred, feed_dict)
            for k in range(len(prediction)):
                labels = get_labels(prediction[k])
                if m < len(test_img_list):
                    img_name = test_img_list[m]
                else:
                    break
                df = df.append({'Image': img_name, 'Id': labels[0] + " " + labels[1] + " " + labels[2] + " " + labels[3] + " " + labels[4]}, ignore_index=True)
                m += 1
        
        df.to_csv('outputs.csv', encoding='utf-8', index=False)

In [7]:
run_model(100)

Epoch: 0, Iter: 0, Loss: 15.6159
Epoch: 0, Iter: 1, Loss: 8.5191
Epoch: 0, Iter: 2, Loss: 7.9355
Epoch: 0, Iter: 3, Loss: 9.8507
Epoch: 0, Iter: 4, Loss: 8.3007
Epoch: 0, Iter: 5, Loss: 8.3794
Epoch: 0, Iter: 6, Loss: 8.4184
Epoch: 0, Iter: 7, Loss: 8.3919
Epoch: 0, Iter: 8, Loss: 8.3280
Epoch: 0, Iter: 9, Loss: 8.4194
Epoch: 0, Iter: 10, Loss: 8.3751
Epoch: 0, Iter: 11, Loss: 8.3314
Epoch: 0, Iter: 12, Loss: 8.3765
Epoch: 0, Iter: 13, Loss: 8.3709
Epoch: 0, Iter: 14, Loss: 8.3819
Epoch: 0, Iter: 15, Loss: 8.3785
Epoch: 0, Iter: 16, Loss: 8.3586
Epoch: 0, Iter: 17, Loss: 8.3716
Epoch: 0, Iter: 18, Loss: 8.3634
Epoch: 0, Iter: 19, Loss: 8.3707
Epoch: 0, Iter: 20, Loss: 8.2584
Epoch: 0, Iter: 21, Loss: 8.3959
Epoch: 0, Iter: 22, Loss: 8.3709
Epoch: 0, Iter: 23, Loss: 8.3969
Epoch: 0, Iter: 24, Loss: 8.3347
Epoch: 0, Iter: 25, Loss: 8.3605
Epoch: 0, Iter: 26, Loss: 8.3289
Epoch: 0, Iter: 27, Loss: 8.3689
Epoch: 0, Iter: 28, Loss: 8.3394
Epoch: 0, Iter: 29, Loss: 8.3917
Epoch: 0, Iter: 30,

Epoch: 1, Iter: 49, Loss: 8.3002
Epoch: 1, Iter: 50, Loss: 8.2953
Epoch: 1, Iter: 51, Loss: 8.4075
Epoch: 1, Iter: 52, Loss: 8.2947
Epoch: 1, Iter: 53, Loss: 8.3010
Epoch: 1, Iter: 54, Loss: 8.3297
Epoch: 1, Iter: 55, Loss: 8.3207
Epoch: 1, Iter: 56, Loss: 8.3145
Epoch: 1, Iter: 57, Loss: 8.3541
Epoch: 1, Iter: 58, Loss: 8.3289
Epoch: 1, Iter: 59, Loss: 8.2880
Epoch: 1, Iter: 60, Loss: 8.3474
Epoch: 1, Iter: 61, Loss: 8.2868
Epoch: 1, Iter: 62, Loss: 8.3044
Epoch: 1, Iter: 63, Loss: 8.4087
Epoch: 1, Iter: 64, Loss: 8.4146
Epoch: 1, Iter: 65, Loss: 8.2431
Epoch: 1, Iter: 66, Loss: 8.3238
Epoch: 1, Iter: 67, Loss: 8.3595
Epoch: 1, Iter: 68, Loss: 8.3836
Epoch: 1, Iter: 69, Loss: 8.2827
Epoch: 1, Iter: 70, Loss: 8.3231
Epoch: 1, Iter: 71, Loss: 8.3244
Epoch: 1, Iter: 72, Loss: 8.3402
Epoch: 1, Iter: 73, Loss: 8.3453
Epoch: 1, Iter: 74, Loss: 8.3516
Epoch: 1, Iter: 75, Loss: 8.3279
Epoch: 1, Iter: 76, Loss: 8.3731
Epoch: 1, Iter: 77, Loss: 8.3466
Epoch: 1, Iter: 78, Loss: 8.3295
Epoch: 1, 

Epoch: 2, Iter: 98, Loss: 8.3017
Epoch: 2, Iter: 99, Loss: 8.2851
Epoch: 2, Iter: 100, Loss: 8.2014
Epoch: 2, Iter: 101, Loss: 8.2539
Epoch: 2, Iter: 102, Loss: 8.3333
Epoch: 2, Iter: 103, Loss: 8.3130
Epoch: 2, Iter: 104, Loss: 8.2806
Epoch: 2, Iter: 105, Loss: 8.2827
Epoch: 2, Iter: 106, Loss: 8.1777
Epoch: 2, Iter: 107, Loss: 8.3589
Epoch: 2, Iter: 108, Loss: 8.2763
Epoch: 2, Iter: 109, Loss: 8.2661
Epoch: 2, Iter: 110, Loss: 8.1802
Epoch: 2, Iter: 111, Loss: 8.2002
Epoch: 2, Iter: 112, Loss: 8.2014
Epoch: 2, Iter: 113, Loss: 8.3386
Epoch: 2, Iter: 114, Loss: 8.2535
Epoch: 2, Iter: 115, Loss: 8.2790
Epoch: 2, Iter: 116, Loss: 8.4140
Epoch: 2, Iter: 117, Loss: 8.2931
Epoch: 2, Iter: 118, Loss: 8.3221
Epoch: 2, Iter: 119, Loss: 8.2834
Epoch: 2, Iter: 120, Loss: 8.2897
Epoch: 2, Iter: 121, Loss: 8.3284
Epoch: 2, Iter: 122, Loss: 8.3060
Epoch: 2, Iter: 123, Loss: 8.2884
Epoch: 2, Iter: 124, Loss: 8.3118
Epoch: 2, Iter: 125, Loss: 8.3147
Epoch: 2, Iter: 126, Loss: 8.2849
Epoch: 2, Iter: 

Epoch: 3, Iter: 146, Loss: 8.1801
Epoch: 3, Iter: 147, Loss: 8.3013
Epoch: 3, Iter: 148, Loss: 8.2322
Epoch: 3, Iter: 149, Loss: 8.3646
Epoch: 3, Iter: 150, Loss: 8.1622
Epoch: 3, Iter: 151, Loss: 8.1048
Epoch: 3, Iter: 152, Loss: 8.2130
Epoch: 3, Iter: 153, Loss: 8.1031
Epoch: 3, Iter: 154, Loss: 8.3343
Epoch: 3, Iter: 155, Loss: 8.1395
Epoch: 3, Iter: 156, Loss: 8.0062
Epoch: 3, Iter: 157, Loss: 8.2624
Epoch: 3, Iter: 158, Loss: 8.2229
Epoch: 3, Iter: 159, Loss: 8.0741
Epoch: 3, Iter: 160, Loss: 8.1873
Epoch: 3, Iter: 161, Loss: 8.3497
Epoch: 3, Iter: 162, Loss: 8.2007
Epoch: 3, Iter: 163, Loss: 8.3238
Epoch: 3, Iter: 164, Loss: 8.2767
Epoch: 3, Iter: 165, Loss: 8.3005
Epoch: 3, Iter: 166, Loss: 8.2039
Epoch: 3, Iter: 167, Loss: 8.2169
Epoch: 3, Iter: 168, Loss: 8.2626
Epoch: 3, Iter: 169, Loss: 8.2068
Epoch: 3, Iter: 170, Loss: 8.1693
Epoch: 3, Iter: 171, Loss: 8.2298
Epoch: 3, Iter: 172, Loss: 8.2540
Epoch: 3, Iter: 173, Loss: 8.1753
Epoch: 3, Iter: 174, Loss: 8.2235
Epoch: 3, Iter

Epoch: 4, Iter: 194, Loss: 8.2397
Epoch: 4, Iter: 195, Loss: 8.2952
Epoch: 4, Iter: 196, Loss: 8.1218
Epoch: 5, Iter: 0, Loss: 8.1398
Epoch: 5, Iter: 1, Loss: 8.1484
Epoch: 5, Iter: 2, Loss: 8.1293
Epoch: 5, Iter: 3, Loss: 8.0617
Epoch: 5, Iter: 4, Loss: 8.1455
Epoch: 5, Iter: 5, Loss: 8.0227
Epoch: 5, Iter: 6, Loss: 8.1094
Epoch: 5, Iter: 7, Loss: 8.0488
Epoch: 5, Iter: 8, Loss: 8.1475
Epoch: 5, Iter: 9, Loss: 7.9033
Epoch: 5, Iter: 10, Loss: 8.0295
Epoch: 5, Iter: 11, Loss: 8.0802
Epoch: 5, Iter: 12, Loss: 8.1484
Epoch: 5, Iter: 13, Loss: 8.0142
Epoch: 5, Iter: 14, Loss: 7.9318
Epoch: 5, Iter: 15, Loss: 8.0145
Epoch: 5, Iter: 16, Loss: 8.0768
Epoch: 5, Iter: 17, Loss: 8.1479
Epoch: 5, Iter: 18, Loss: 8.3412
Epoch: 5, Iter: 19, Loss: 8.1634
Epoch: 5, Iter: 20, Loss: 8.0842
Epoch: 5, Iter: 21, Loss: 8.1471
Epoch: 5, Iter: 22, Loss: 8.0331
Epoch: 5, Iter: 23, Loss: 8.1333
Epoch: 5, Iter: 24, Loss: 7.9880
Epoch: 5, Iter: 25, Loss: 8.2440
Epoch: 5, Iter: 26, Loss: 7.9419
Epoch: 5, Iter: 2

...

Epoch: 94, Iter: 139, Loss: 2.1774
Epoch: 94, Iter: 140, Loss: 2.5312
Epoch: 94, Iter: 141, Loss: 2.5589
Epoch: 94, Iter: 142, Loss: 2.0091
Epoch: 94, Iter: 143, Loss: 3.0244
Epoch: 94, Iter: 144, Loss: 2.3560
Epoch: 94, Iter: 145, Loss: 2.0347
Epoch: 94, Iter: 146, Loss: 2.5175
Epoch: 94, Iter: 147, Loss: 2.5076
Epoch: 94, Iter: 148, Loss: 1.8414
Epoch: 94, Iter: 149, Loss: 2.7159
Epoch: 94, Iter: 150, Loss: 1.6728
Epoch: 94, Iter: 151, Loss: 0.5153
Epoch: 94, Iter: 152, Loss: 2.6805
Epoch: 94, Iter: 153, Loss: 2.0055
Epoch: 94, Iter: 154, Loss: 1.6733
Epoch: 94, Iter: 155, Loss: 1.3776
Epoch: 94, Iter: 156, Loss: 1.1933
Epoch: 94, Iter: 157, Loss: 2.0846
Epoch: 94, Iter: 158, Loss: 1.5048
Epoch: 94, Iter: 159, Loss: 1.5052
Epoch: 94, Iter: 160, Loss: 2.4825
Epoch: 94, Iter: 161, Loss: 2.6760
Epoch: 94, Iter: 162, Loss: 1.5205
Epoch: 94, Iter: 163, Loss: 2.1836
Epoch: 94, Iter: 164, Loss: 2.0192
Epoch: 94, Iter: 165, Loss: 3.0116
Epoch: 94, Iter: 166, Loss: 2.1760
Epoch: 94, Iter: 167

Epoch: 95, Iter: 180, Loss: 1.8466
Epoch: 95, Iter: 181, Loss: 1.6790
Epoch: 95, Iter: 182, Loss: 1.5064
Epoch: 95, Iter: 183, Loss: 2.2787
Epoch: 95, Iter: 184, Loss: 1.8465
Epoch: 95, Iter: 185, Loss: 2.2095
Epoch: 95, Iter: 186, Loss: 1.1718
Epoch: 95, Iter: 187, Loss: 1.4089
Epoch: 95, Iter: 188, Loss: 1.1775
Epoch: 95, Iter: 189, Loss: 2.6937
Epoch: 95, Iter: 190, Loss: 1.8421
Epoch: 95, Iter: 191, Loss: 1.3405
Epoch: 95, Iter: 192, Loss: 1.1737
Epoch: 95, Iter: 193, Loss: 2.0060
Epoch: 95, Iter: 194, Loss: 1.9201
Epoch: 95, Iter: 195, Loss: 2.5071
Epoch: 95, Iter: 196, Loss: 2.1788
Epoch: 96, Iter: 0, Loss: 2.2451
Epoch: 96, Iter: 1, Loss: 2.0091
Epoch: 96, Iter: 2, Loss: 2.3411
Epoch: 96, Iter: 3, Loss: 2.0964
Epoch: 96, Iter: 4, Loss: 2.3930
Epoch: 96, Iter: 5, Loss: 2.4346
Epoch: 96, Iter: 6, Loss: 1.5086
Epoch: 96, Iter: 7, Loss: 1.6721
Epoch: 96, Iter: 8, Loss: 1.6750
Epoch: 96, Iter: 9, Loss: 1.6947
Epoch: 96, Iter: 10, Loss: 1.6882
Epoch: 96, Iter: 11, Loss: 1.6718
Epoch: 

Epoch: 97, Iter: 25, Loss: 2.6885
Epoch: 97, Iter: 26, Loss: 1.2474
Epoch: 97, Iter: 27, Loss: 1.6845
Epoch: 97, Iter: 28, Loss: 2.5073
Epoch: 97, Iter: 29, Loss: 1.6779
Epoch: 97, Iter: 30, Loss: 2.5274
Epoch: 97, Iter: 31, Loss: 2.0080
Epoch: 97, Iter: 32, Loss: 2.2257
Epoch: 97, Iter: 33, Loss: 3.1914
Epoch: 97, Iter: 34, Loss: 2.7637
Epoch: 97, Iter: 35, Loss: 1.1704
Epoch: 97, Iter: 36, Loss: 2.5157
Epoch: 97, Iter: 37, Loss: 2.6810
Epoch: 97, Iter: 38, Loss: 3.0560
Epoch: 97, Iter: 39, Loss: 3.0099
Epoch: 97, Iter: 40, Loss: 2.5338
Epoch: 97, Iter: 41, Loss: 3.5197
Epoch: 97, Iter: 42, Loss: 2.5543
Epoch: 97, Iter: 43, Loss: 2.2012
Epoch: 97, Iter: 44, Loss: 2.3468
Epoch: 97, Iter: 45, Loss: 2.8771
Epoch: 97, Iter: 46, Loss: 1.5412
Epoch: 97, Iter: 47, Loss: 2.1727
Epoch: 97, Iter: 48, Loss: 2.0275
Epoch: 97, Iter: 49, Loss: 1.8712
Epoch: 97, Iter: 50, Loss: 1.3415
Epoch: 97, Iter: 51, Loss: 2.1919
Epoch: 97, Iter: 52, Loss: 2.1786
Epoch: 97, Iter: 53, Loss: 2.5111
Epoch: 97, Ite

Epoch: 98, Iter: 67, Loss: 2.2274
Epoch: 98, Iter: 68, Loss: 1.6734
Epoch: 98, Iter: 69, Loss: 2.6787
Epoch: 98, Iter: 70, Loss: 2.3797
Epoch: 98, Iter: 71, Loss: 1.6738
Epoch: 98, Iter: 72, Loss: 2.9626
Epoch: 98, Iter: 73, Loss: 1.8497
Epoch: 98, Iter: 74, Loss: 1.9307
Epoch: 98, Iter: 75, Loss: 1.6874
Epoch: 98, Iter: 76, Loss: 2.3689
Epoch: 98, Iter: 77, Loss: 1.5266
Epoch: 98, Iter: 78, Loss: 2.2209
Epoch: 98, Iter: 79, Loss: 2.0091
Epoch: 98, Iter: 80, Loss: 1.6749
Epoch: 98, Iter: 81, Loss: 2.5551
Epoch: 98, Iter: 82, Loss: 1.6954
Epoch: 98, Iter: 83, Loss: 2.3650
Epoch: 98, Iter: 84, Loss: 3.1204
Epoch: 98, Iter: 85, Loss: 2.0336
Epoch: 98, Iter: 86, Loss: 1.7968
Epoch: 98, Iter: 87, Loss: 2.5239
Epoch: 98, Iter: 88, Loss: 2.6758
Epoch: 98, Iter: 89, Loss: 3.0212
Epoch: 98, Iter: 90, Loss: 1.0092
Epoch: 98, Iter: 91, Loss: 1.6754
Epoch: 98, Iter: 92, Loss: 1.5089
Epoch: 98, Iter: 93, Loss: 2.0526
Epoch: 98, Iter: 94, Loss: 2.3170
Epoch: 98, Iter: 95, Loss: 1.5120
Epoch: 98, Ite

Epoch: 99, Iter: 109, Loss: 2.8456
Epoch: 99, Iter: 110, Loss: 1.8751
Epoch: 99, Iter: 111, Loss: 2.7397
Epoch: 99, Iter: 112, Loss: 1.5068
Epoch: 99, Iter: 113, Loss: 2.6882
Epoch: 99, Iter: 114, Loss: 2.3811
Epoch: 99, Iter: 115, Loss: 1.4273
Epoch: 99, Iter: 116, Loss: 3.7448
Epoch: 99, Iter: 117, Loss: 2.5123
Epoch: 99, Iter: 118, Loss: 2.3492
Epoch: 99, Iter: 119, Loss: 2.2116
Epoch: 99, Iter: 120, Loss: 1.8391
Epoch: 99, Iter: 121, Loss: 2.8581
Epoch: 99, Iter: 122, Loss: 1.8409
Epoch: 99, Iter: 123, Loss: 2.0080
Epoch: 99, Iter: 124, Loss: 1.8406
Epoch: 99, Iter: 125, Loss: 1.3820
Epoch: 99, Iter: 126, Loss: 1.8441
Epoch: 99, Iter: 127, Loss: 1.5048
Epoch: 99, Iter: 128, Loss: 3.0086
Epoch: 99, Iter: 129, Loss: 2.7478
Epoch: 99, Iter: 130, Loss: 3.0738
Epoch: 99, Iter: 131, Loss: 3.6354
Epoch: 99, Iter: 132, Loss: 2.7168
Epoch: 99, Iter: 133, Loss: 1.3923
Epoch: 99, Iter: 134, Loss: 2.1254
Epoch: 99, Iter: 135, Loss: 1.5049
Epoch: 99, Iter: 136, Loss: 1.5071
Epoch: 99, Iter: 137